In [1]:
#### Start a Spark Session
from pyspark.sql import SparkSession
import os
# import sys

# os.environ['SPARK_HOME'] = "/Users/mario/anaconda3/envs/streaming_app/lib/python3.7/site-packages/pyspark"

# mysql driver for java
SUBMIT_ARGS = "--jars /Users/mario/anaconda3/envs/streaming_app/lib/python3.7/site-packages/pyspark/jars/mysql-connector-java-8.0.13.jar pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

# beginning a spark session to be able to use spark and enabling Hive support to be able to create a database table from the csv schema
spark = SparkSession.builder.appName("streaming_app").enableHiveSupport().getOrCreate()

In [2]:
# bringing in ~7GB csv to read in spark
df = spark.read.csv("data (5).csv", header=True)
# df.schema
df2 = df
# Removing all spaces from column names
for col in df.columns:
    df2 = df2.withColumnRenamed(col,col.replace(" ", ""))
    
# Renaming column name with byte size of 64 bytes to 58 bytes because was getting error 'Identifier name  is too long'
# Every other column name gave no errors
df2 = df2.withColumnRenamed("ProviderBusinessPracticeLocationAddressCountryCode(IfoutsideU.S.)", "ProviderBusinessPracticeLocationCountryCode(IfoutsideU.S.)")

# schema that we will be using to build our mysql table from our csv
userSchema = df2.schema

# column names from csv file
df2.columns  

['NPI',
 'EntityTypeCode',
 'ReplacementNPI',
 'EmployerIdentificationNumber(EIN)',
 'ProviderOrganizationName(LegalBusinessName)',
 'ProviderLastName(LegalName)',
 'ProviderFirstName',
 'ProviderMiddleName',
 'ProviderNamePrefixText',
 'ProviderNameSuffixText',
 'ProviderCredentialText',
 'ProviderOtherOrganizationName',
 'ProviderOtherOrganizationNameTypeCode',
 'ProviderOtherLastName',
 'ProviderOtherFirstName',
 'ProviderOtherMiddleName',
 'ProviderOtherNamePrefixText',
 'ProviderOtherNameSuffixText',
 'ProviderOtherCredentialText',
 'ProviderOtherLastNameTypeCode',
 'ProviderFirstLineBusinessMailingAddress',
 'ProviderSecondLineBusinessMailingAddress',
 'ProviderBusinessMailingAddressCityName',
 'ProviderBusinessMailingAddressStateName',
 'ProviderBusinessMailingAddressPostalCode',
 'ProviderBusinessMailingAddressCountryCode(IfoutsideU.S.)',
 'ProviderBusinessMailingAddressTelephoneNumber',
 'ProviderBusinessMailingAddressFaxNumber',
 'ProviderFirstLineBusinessPracticeLocationAddr

In [3]:
# checking to see what the schema looks like
userSchema

StructType(List(StructField(NPI,StringType,true),StructField(EntityTypeCode,StringType,true),StructField(ReplacementNPI,StringType,true),StructField(EmployerIdentificationNumber(EIN),StringType,true),StructField(ProviderOrganizationName(LegalBusinessName),StringType,true),StructField(ProviderLastName(LegalName),StringType,true),StructField(ProviderFirstName,StringType,true),StructField(ProviderMiddleName,StringType,true),StructField(ProviderNamePrefixText,StringType,true),StructField(ProviderNameSuffixText,StringType,true),StructField(ProviderCredentialText,StringType,true),StructField(ProviderOtherOrganizationName,StringType,true),StructField(ProviderOtherOrganizationNameTypeCode,StringType,true),StructField(ProviderOtherLastName,StringType,true),StructField(ProviderOtherFirstName,StringType,true),StructField(ProviderOtherMiddleName,StringType,true),StructField(ProviderOtherNamePrefixText,StringType,true),StructField(ProviderOtherNameSuffixText,StringType,true),StructField(ProviderOth

In [4]:
# creating a spark dataframe from the csv being read
readDf = spark.read.format("csv").schema(userSchema).load("data (5).csv")
readDf.show()

+----------+----------------+---------------+---------------------------------+-------------------------------------------+---------------------------+-------------------+--------------------+----------------------+----------------------+----------------------+-----------------------------+-------------------------------------+---------------------+----------------------+-----------------------+---------------------------+---------------------------+---------------------------+-----------------------------+---------------------------------------+----------------------------------------+--------------------------------------+---------------------------------------+----------------------------------------+--------------------------------------------------------+---------------------------------------------+---------------------------------------+------------------------------------------------+-------------------------------------------------+---------------------------------------------

In [5]:
# creating a temp table to be able to create a hive table
readDf.createOrReplaceTempView("temp_npidatatable")

In [ ]:
spark.sql("create table npi_data as select * from temp_npidatatable")

In [11]:
# mysql db credentials
username = "root"
password = ""
jdbcHostname = "127.0.0.1"
jdbcDatabase = "pyspark_db"
jdbcPort = 3306
jdbcUrl = "jdbc:mysql://{0}:{1}/{2}?user={3}&password={4}".format(jdbcHostname, jdbcPort, jdbcDatabase, username, password)
connectionProperties = {
  "user" : username,
  "password" : password,
  "driver" : "com.mysql.jdbc.Driver"
}

# using our Hive table npi_data to create a mysql table 
spark.table("npi_data").write.jdbc(jdbcUrl, "npi_data_table", "overwrite", connectionProperties)

# using this example as reference only
# df.write.jdbc("jdbc:sqlserver://serverURL", "dbName.SparkTestTable1", "overwrite", dict(driver="com.microsoft.sqlserver.jdbc.SQLServerDriver", user="userName", password="password"))

Py4JJavaError: An error occurred while calling o628.jdbc.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 20, localhost, executor driver): java.sql.BatchUpdateException: Row size too large (> 8126). Changing some columns to TEXT or BLOB may help. In current row format, BLOB prefix of 0 bytes is stored inline.
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at com.mysql.cj.util.Util.handleNewInstance(Util.java:191)
	at com.mysql.cj.util.Util.getInstance(Util.java:166)
	at com.mysql.cj.util.Util.getInstance(Util.java:173)
	at com.mysql.cj.jdbc.exceptions.SQLError.createBatchUpdateExceptio